In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def xavier_init(in_shape):
  val = tf.random_normal(shape = in_shape, stddev = 1./tf.sqrt(in_shape[0]/2.))
  return val

In [ ]:
learning_rate = 0.002
batch_size = 32
epochs = 100000
Z_dim = 64
image_dimension = 784

D_H1 = 256
G_H1 = 256
E_H1 = 256

In [ ]:

Disc_W = {"disc_H" : tf.Variable(xavier_init([image_dimension + Z_dim, D_H1])),
          "disc_final" : tf.Variable(xavier_init([D_H1, 1]))
          }

Disc_Bias = {"disc_H" : tf.Variable(xavier_init([D_H1])),
          "disc_final" : tf.Variable(xavier_init([1]))
          }

In [ ]:

Gen_W = {"Gen_H" : tf.Variable(xavier_init([Z_dim, G_H1])),
          "Gen_final" : tf.Variable(xavier_init([G_H1, image_dimension]))
          }

Gen_Bias = {"Gen_H" : tf.Variable(xavier_init([G_H1])),
          "Gen_final" : tf.Variable(xavier_init([image_dimension]))
          }

In [ ]:
E_W = {"E_H" : tf.Variable(xavier_init([image_dimension, E_H1])),
          "E_final" : tf.Variable(xavier_init([E_H1, Z_dim]))
          }

E_Bias = {"E_H" : tf.Variable(xavier_init([E_H1])),
          "E_final" : tf.Variable(xavier_init([Z_dim]))
          }

In [ ]:
X = tf.placeholder(tf.float32, shape = [None, image_dimension])
Z = tf.placeholder(tf.float32, shape = [None, Z_dim])

In [ ]:
def Z_distribution(m, n):
  return np.random.uniform(-1., 1., size = [m, n])

In [ ]:
def Generator(z):  
  G_hidden_layer = tf.nn.relu(tf.add(tf.matmul(z, Gen_W['Gen_H']), Gen_Bias['Gen_H']))
  gen_output = tf.add(tf.matmul(G_hidden_layer, Gen_W['Gen_final']), Gen_Bias['Gen_final'])
  gen_prob_output = tf.nn.sigmoid(gen_output)
  return gen_prob_output

In [ ]:
def Discriminator(x,z):
  input = tf.concat(axis = 1, values = [x, z])  
  D_hidden_layer = tf.nn.relu(tf.add(tf.matmul(input, Disc_W['disc_H']), Disc_Bias['disc_H']))
  disc_output = tf.add(tf.matmul(D_hidden_layer, Disc_W['disc_final']), Disc_Bias['disc_final'])
  disc_prob_output = tf.nn.sigmoid(disc_output)
  return disc_prob_output

In [ ]:
def Encoder_NN(x):
  Q_hidden_layer = tf.nn.relu(tf.add(tf.matmul(x, Q_W['Q_H']), Q_Bias['Q_H']))
  Q_output = tf.add(tf.matmul(Q_hidden_layer, Q_W['Q_final']), Q_Bias['Q_final'])
  Q_prob_output = (Q_output)
  return Q_prob_output

In [ ]:
z_generated = Encoder_NN(X)
x_generated = Generator(Z)

real_output_Disc = Discriminator(X, z_generated)
fake_output_Disc = Discriminator(x_generated, Z)

In [ ]:
Disc_Loss = -tf.reduce_mean(tf.log(real_output_Disc + 1e-7) + tf.log(1.0 - fake_output_Disc + 1e-7))
Gen_Loss = -tf.reduce_mean(tf.log(fake_output_Disc + 1e-7) + tf.log(1.0 - real_output_Disc + 1e-7))

In [ ]:
Disc_params = [Disc_W['disc_H'], Disc_W['disc_final'], Disc_Bias['disc_H'], Disc_Bias['disc_final']]
Gen_params = [Gen_W['Gen_H'], Gen_W['Gen_final'], Gen_Bias['Gen_H'], Gen_Bias['Gen_final']]
E_params = [E_W['E_H'], E_W['E_final'], E_Bias['E_H'], E_Bias['E_final']]

In [ ]:
gen_optimize = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(Gen_Loss, var_list = Gen_params + E_params)
Disc_optimize = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(Disc_Loss, var_list = Disc_params)


In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for epoch in range(epochs):
  X_batch, _ = mnist.train.next_batch(batch_size)
  z_noise = Z_distribution(batch_size, Z_dim)

  _, Disc_loss_epoch = sess.run([Disc_optimize, Disc_Loss], feed_dict = {X:X_batch, Z:z_noise})
  _, Gen_loss_epoch = sess.run([gen_optimize, Gen_Loss], feed_dict = {X:X_batch, Z:z_noise})


  if epoch%2000 == 0:
    print('Steps: {0}, Disc Loss: {1}, Generator Loss: {2}'.format(epoch, Disc_loss_epoch, Gen_loss_epoch))

In [ ]:
test_output , _ = Generator(Z)
n = 6
canvas = np.empty((28*n, 28*n))

for i in range(n):
  Z_noise = Z_distribution(batch_size, Z_dim)

  generated_image  = sess.run(test_output, feed_dict = {Z:Z_noise})
  for j in range(n):
    canvas[i * 28: (i+1) * 28, j * 28: (j+1) * 28] = generated_image[j].reshape([28, 28])


plt.figure(figsize = (n, n))
plt.imshow(canvas, origin = "upper", cmap = 'gray')
plt.show()

In [ ]:
#Validation that the enc and the decoder are inverse of each other, by passing one sample through the whole cycle

z_generated = Encoder_NN(X)
test_output = Generator(z_generated)
n = 6
canvas1 = np.empty((28*n, 28*n))
canvas2 = np.empty((28*n, 28*n))

for i in range(n):
  X_batch, _ = mnist.train.next_batch(batch_size)

  generated_image  = sess.run(test_output, feed_dict = {X:X_batch})
  for j in range(n):
    canvas2[i * 28: (i+1) * 28, j * 28: (j+1) * 28] = generated_image[j].reshape([28, 28])

  for j in range(n):
    canvas1[i * 28: (i+1) * 28, j * 28: (j+1) * 28] = X_batch[j].reshape([28, 28])


f, ax = plt.subplots(1, 2)
ax[0].imshow(canvas1, origin = "upper", cmap = 'gray')
ax[1].imshow(canvas2, origin = "upper", cmap = 'gray')
plt.show()